In [1]:
import pandas as pd
import numpy as np
import os
import uuid
import torch

In [2]:
df_list01 = []
df_list02 = []
final_df_list01 = []
final_df_list02 = []
for root, directories, files in os.walk('./DataSet'):
    for file in files:
        if file.endswith('right.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['procedure'] = 'network01'
            df_list01.append(df)
        elif file.endswith('left.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['procedure'] = 'network02'
            df_list02.append(df)
        elif file.endswith('labels.csv'):
            df = pd.read_csv(os.path.join(root, file))
            df['dataset'] = root.rsplit('/', 1)[-1]
            df_list01.append(df)
            df_list02.append(df)
    if df_list01:
         combined_df01 = pd.concat(df_list01, axis=1)
         final_df_list01.append(combined_df01)
         df_list01.clear()
    else:
         combined_df = pd.DataFrame()

    if df_list02:
         combined_df02 = pd.concat(df_list02, axis=1)
         final_df_list02.append(combined_df02)
         df_list02.clear()
    else:
         combined_df = pd.DataFrame()

final_df01 = pd.concat(final_df_list01, axis=0)
final_df02 = pd.concat(final_df_list02, axis=0)

final_df = pd.concat([final_df01, final_df02], axis=0)

new_column_names = {
    'temp_dashboard': 'temp_t_dashboard',
    'temp_above_suspension': 'temp_t_above_suspension',
    'temp_below_suspension': 'temp_t_below_suspension'
}
final_df.rename(columns=new_column_names, inplace=True)


Road_Surface_Condition = final_df[['paved_road', 'unpaved_road']]
Road_Surface_Type_Labels = final_df[['dirt_road', 'cobblestone_road', 'asphalt_road']]
Road_Roughness_Condition_Left = final_df[['good_road_left', 'regular_road_left','bad_road_left']]
Road_Roughness_Condition_Right = final_df[['good_road_right', 'regular_road_right','bad_road_right']]
Speed_Bump_Types= final_df[['no_speed_bump', 'speed_bump_asphalt','speed_bump_cobblestone']]
final_df['Road_Surface_Type_Labels'] = pd.Series(Road_Surface_Type_Labels.columns[np.where(Road_Surface_Type_Labels !=0)[1]])
final_df['Road_Surface_Condition'] = pd.Series(Road_Surface_Condition.columns[np.where(Road_Surface_Condition !=0)[1]])
final_df['Road_Roughness_Condition_Left'] = pd.Series(Road_Roughness_Condition_Left.columns[np.where(Road_Roughness_Condition_Left !=0)[1]])
final_df['Road_Roughness_Condition_Right'] = pd.Series(Road_Roughness_Condition_Right.columns[np.where(Road_Roughness_Condition_Right !=0)[1]])
final_df['Speed_Bump_Types'] = pd.Series(Speed_Bump_Types.columns[np.where(Speed_Bump_Types !=0)[1]])
final_df = final_df.drop(columns=['paved_road', 'unpaved_road','dirt_road', 'cobblestone_road', 'asphalt_road','good_road_left', 'regular_road_left','bad_road_left',                                      'good_road_right', 'regular_road_right','bad_road_right', 'no_speed_bump', 'speed_bump_asphalt','speed_bump_cobblestone'])
final_df['unique_id'] = [str(uuid.uuid4())[:8] for _ in range(len(final_df))]
print(final_df.columns)

Index(['timestamp', 'acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension', 'timestamp_gps', 'latitude', 'longitude',
       'speed', 'procedure', 'dataset', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types', 'unique_id'],
      dtype='ob

In [3]:
data_collection_contexts = pd.read_csv('./DataSet/data_collection_contexts.csv')
PVS_Dataset_complete  = final_df.set_index('dataset').join(data_collection_contexts.set_index('DataSet'))
shuffled_df = PVS_Dataset_complete.sample(frac=1, random_state=42)
PVS_Dataset = shuffled_df[:10000]
print(PVS_Dataset.columns)

Index(['timestamp', 'acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension', 'timestamp_gps', 'latitude', 'longitude',
       'speed', 'procedure', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types', 'unique_id',
       'Unnamed: 0', 'Vehic

In [4]:
property = ['acc_x_dashboard', 'acc_y_dashboard', 'acc_z_dashboard',
       'acc_x_above_suspension', 'acc_y_above_suspension',
       'acc_z_above_suspension', 'acc_x_below_suspension',
       'acc_y_below_suspension', 'acc_z_below_suspension', 'gyro_x_dashboard',
       'gyro_y_dashboard', 'gyro_z_dashboard', 'gyro_x_above_suspension',
       'gyro_y_above_suspension', 'gyro_z_above_suspension',
       'gyro_x_below_suspension', 'gyro_y_below_suspension',
       'gyro_z_below_suspension', 'mag_x_dashboard', 'mag_y_dashboard',
       'mag_z_dashboard', 'mag_x_above_suspension', 'mag_y_above_suspension',
       'mag_z_above_suspension', 'temp_t_dashboard', 'temp_t_above_suspension',
       'temp_t_below_suspension']

labels = []
table_list = []
for column_name in PVS_Dataset.columns:
    if column_name in property:
        column_values = PVS_Dataset[column_name].values.tolist()

        new_df = pd.DataFrame({'measure': column_values})

        column_name_split = column_name.split('_')
        if len(column_name_split) == 3 :
            new_df['sensor'] = column_name_split[0]
            new_df['property'] = '_'.join(column_name_split[:2])
            new_df['feature'] = column_name_split[-1]
            if column_name_split[0] == "mag":
                new_df['platform'] = "AK8963"
            else:
                new_df['platform'] = 'MPU-6500'
        elif len(column_name_split) == 4 :
            new_df['sensor'] = column_name_split[0]
            new_df['property'] = '_'.join(column_name_split[:2])
            new_df['feature'] = '_'.join(column_name_split[2:])
            if column_name_split[0] == "mag":
                new_df['platform'] = "AK8963"
            else:
                new_df['platform'] = 'MPU-6500'

        new_df['timestamp_gps'] = PVS_Dataset['timestamp_gps'].to_numpy()
        new_df['vehicle'] = PVS_Dataset['Vehicle'].to_numpy()
        new_df['procedure'] = PVS_Dataset['procedure'].to_numpy()
        new_df['driver'] = PVS_Dataset['Driver'].to_numpy()
        new_df['scenario'] = PVS_Dataset['Scenario'].to_numpy()
        new_df['latitude'] = PVS_Dataset['latitude'].to_numpy()
        new_df['longitude'] = PVS_Dataset['longitude'].to_numpy()
        new_df['Distance'] = PVS_Dataset['Scenario'].to_numpy()
        new_df['datetime'] = PVS_Dataset['timestamp'].to_numpy()
        new_df['unique_id'] = PVS_Dataset['unique_id'].to_numpy()
        new_df['Road_Surface_Type_Labels'] = PVS_Dataset['Road_Surface_Type_Labels'].to_numpy()
        new_df['Road_Surface_Condition'] = PVS_Dataset['Road_Surface_Condition'].to_numpy()
        new_df['Road_Roughness_Condition_Left'] = PVS_Dataset['Road_Roughness_Condition_Left'].to_numpy()
        new_df['Road_Roughness_Condition_Right'] = PVS_Dataset['Road_Roughness_Condition_Right'].to_numpy()
        new_df['Speed_Bump_Types'] = PVS_Dataset['Speed_Bump_Types'].to_numpy()
        table_list.append(new_df)

features_table = pd.concat(table_list, axis=0)
features_table.to_csv('./outputs/rowData/features.csv')
print(features_table.columns)

Index(['measure', 'sensor', 'property', 'feature', 'platform', 'timestamp_gps',
       'vehicle', 'procedure', 'driver', 'scenario', 'latitude', 'longitude',
       'Distance', 'datetime', 'unique_id', 'Road_Surface_Type_Labels',
       'Road_Surface_Condition', 'Road_Roughness_Condition_Left',
       'Road_Roughness_Condition_Right', 'Speed_Bump_Types'],
      dtype='object')


In [ ]:
import morph_kgc
graph = morph_kgc.materialize("./mapping/config.ini")
graph.serialize(destination="./outputs/triples/result.txt")

INFO | 2023-07-09 12:28:21,072 | Parallelization is not supported for darwin when running as a library. If you need to speed up your data integration pipeline, please run through the command line.
INFO | 2023-07-09 12:28:21,447 | 60 mapping rules retrieved.
INFO | 2023-07-09 12:28:21,470 | Mapping partition with 60 groups generated.
INFO | 2023-07-09 12:28:21,471 | Maximum number of rules within mapping group: 1.
INFO | 2023-07-09 12:28:21,471 | Mappings processed in 0.397 seconds.
INFO | 2023-07-09 12:28:49,115 | Number of triples generated in total: 1655440.


In [ ]:
from rdflib import Graph, store

g = Graph(store='neo4j-cypher')
theconfig = {'uri': "bolt://localhost:7687", 'database': 'neo4j', 'auth': {'user': "neo4j", 'pwd': "28071382"}}

g.open(theconfig, create = False)

In [ ]:
g.store.startBatchedWrite()
g.parse("./outputs/triples/result.txt", format="ttl")
g.store.endBatchedWrite()